Purpose of this notebook is to experiment with how the explainations are created.
In section 1, we first initialize functions for (loading llm, connecting vector db, and en embeddings model)
In section 2, we experiment and run this flow.

# Initialize

In [ ]:
# install dependencies
!pip install sentence_transformers 
!pip install qdrant_client
!pip install einops

In [1]:
# import libraries
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams, Batch
from collections import Counter
import requests
import json
import os
from tqdm import tqdm
import pandas as pd
import json

# Change to the parent directory of the notebook
os.chdir('..')

## 

## Load Embeddings model

If you are just experimenting with explainations and here to run LLM. then no need to run the cells in this section.

Bellow there are 3 cells with 3 different type of models. uncomment the one suits best for you

In [2]:
# RUNS superfast on CPU, Bad Results, good for old or weak laptop cpus, speed up testing

# EMBEDDINGS_MODEL = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2', trust_remote_code=True)

In [ ]:
# RUNS superfast on CPU too, Good Results, works well on laptop with good cpu and laptop without GPU.

EMBEDDINGS_MODEL = SentenceTransformer('jxm/cde-small-v1', trust_remote_code=True, device='cpu')

In [9]:
# ONLY USE THIS IF YOU HAVE CUDA (NVIDIA GPU with 9 GB VRAM available:
# RUNS superfast on GPU, Best Results.

# EMBEDDINGS_MODEL = SentenceTransformer("dunzhang/stella_en_1.5B_v5", trust_remote_code=True, device='cpu') #.cuda()

In [7]:
%%time

# benchmarking how much time it takes to create embeddings of 50 iters
for i in range(50):
    EMBEDDINGS_MODEL.encode("# testing how much time embeddings model take on your system")

CPU times: total: 17.1 s
Wall time: 2.17 s


In [8]:
# dimension
len(EMBEDDINGS_MODEL.encode(""))

768

## Configure Vector DB

In [13]:
# https://7ef18c4d-2ef6-4fb0-9243-0ac62546593c.us-east4-0.gcp.cloud.qdrant.io:6333/dashboard#/collections

In [14]:
from qdrant_client import QdrantClient

qdrant_client = QdrantClient(
    url="https://7ef18c4d-2ef6-4fb0-9243-0ac62546593c.us-east4-0.gcp.cloud.qdrant.io:6333", 
    api_key="BR8zsNr5lEYrqJPL4EknUj2oRska2JO1nHwPFawlFMqZIrYMuGZ0Wg",
)

print(qdrant_client.get_collections())

collections=[CollectionDescription(name='test_index_'), CollectionDescription(name='HateXplain_index_3'), CollectionDescription(name='HateXplain_index_1'), CollectionDescription(name='HateXplain_index_2'), CollectionDescription(name='HateXplain_index'), CollectionDescription(name='HateXplain_8129'), CollectionDescription(name='HateXplain_index_4'), CollectionDescription(name='test_index')]


In [15]:
# docker only 
qdrant_client = QdrantClient(location='127.0.0.1', port=6333)

In [16]:
def create_index(index_name):
    qdrant_client.create_collection(
        collection_name=index_name,
        vectors_config=VectorParams(size=len(EMBEDDINGS_MODEL.encode("")), distance=Distance.DOT)
    )

## Configure LLM

In [17]:
API_URL = "https://inf.cl.uni-trier.de/chat/"

def llm(model_name, system_prompt, input_query):
    # Construct the request payload
    payload = {
        "messages": [
            {"content": system_prompt, "role": "system"},
            {"content": input_query, "role": "user"}
        ],
        "model": model_name,
        "options": {}
    }
    
    # Set the request headers
    headers = {
        "accept": "application/json",
        "Content-Type": "application/json"
    }
    
    # Send the POST request
    response = requests.post(API_URL, headers=headers, data=json.dumps(payload))
    
    # Process the response
    if response.status_code == 200:
        print("Response received successfully:")
        response = response.json() # json.dumps(, indent=4)
    else:
        print(f"Failed to retrieve response. Status code: {response.status_code}")

    return response


## Load Dataset and convert to DF | Transform

In [18]:
# Load the dataset.json file
file_path = 'Data/dataset.json'
with open(file_path, 'r') as file:
    data = json.load(file)


In [19]:

rows = []
for row in data.items():
    post_id = row[1]['post_id']
    post_tokens = row[1]['post_tokens']
    rationales = row[1]['rationales']
    annotators = row[1]['annotators']
    tweet_text = " ".join(post_tokens)

    # Aggregate all rationales
    combined_rationale = [any(rat[i] for rat in rationales) for i in range(len(post_tokens))] if rationales else [0] * len(post_tokens)

    # Extract key features
    key_features = [post_tokens[i] for i, is_key in enumerate(combined_rationale) if is_key]

    annotator_data = {}
    all_targets = set()  # To store unique targets
    labels = []  # To collect labels for voting

    for index, annotator in enumerate(annotators, start=1):  # Use index as annotator_id
        annotator_data[f"annotator_{index}_label"] = annotator['label']
        annotator_data[f"annotator_{index}_target"] = ", ".join(annotator['target']) if annotator['target'] else None

        # Collect all targets
        all_targets.update(annotator['target'])

        # Collect labels for voting
        labels.append(annotator['label'])

    # Combine all unique targets
    combined_target = ", ".join(sorted(all_targets))

    # Perform majority voting for the label
    label_counts = Counter(labels)
    voted_label = label_counts.most_common(1)[0][0] if labels else None

    # Create the row
    row_data = {
        "post_id": post_id,
        "tweet_text": tweet_text,
        "key_features": key_features,
        "target": combined_target,
        "label": voted_label,
    }
    row_data.update(annotator_data)  # Add annotator-related fields

    rows.append(row_data)
df = pd.DataFrame(rows)

df.tail()

,post_id,tweet_text,key_features,target,label,annotator_1_label,annotator_1_target,annotator_2_label,annotator_2_target,annotator_3_label,annotator_3_target
20143,9989999_gab,if ur still on twitter tell carlton i said his...,"[alcoholic, drug, addicted, teenage, single, m...","Men, None, Other, Women",offensive,offensive,"Men, Women, Other",offensive,None,normal,None
20144,9990225_gab,when i first got on here and said i hate trump...,"[i, hate, trump, i, was, hating, niggers]","African, None, Other",offensive,offensive,African,offensive,"African, Other",normal,None
20145,9991681_gab,was macht der moslem wenn der zion gegen seine...,[],"Islam, None, Other",normal,offensive,Islam,normal,Other,normal,None
20146,9992513_gab,it is awful look at world demographics asians ...,"[asians, are, fucking, everywhere, white, geno...","Asian, Hispanic",hatespeech,hatespeech,Hispanic,hatespeech,Asian,offensive,Asian
20147,9998729_gab,the jewish globalist elite have only imported ...,"[imported, few, million, muslims, to, multicul...","African, Islam, Jewish",offensive,hatespeech,"African, Islam",offensive,"Islam, Jewish",offensive,"African, Islam, Jewish"


In [20]:
df.to_csv("Data/dataset.csv", index=True)

In [21]:
# filder data
data = [
    {key: d[key] for key in ('post_id', 'tweet_text', 'key_features', 'target', 'label')}
    for d in rows
]

In [24]:
# see data
data[10000]

{'post_id': '1178534874602496000_twitter',
 'tweet_text': '<user> <user> this is exactly the same as the argument that people who get raped when they were dressed scantily were asking for it you are lame af',
 'key_features': [],
 'target': 'None',
 'label': 'normal'}

# Experiments

Here we make experiments with system prompt, 

System prompt contains : what model is supposed to do with the input query. with an exmple output.
Input Query contains, one row of dataset.
explaination variable will have the output of model which will be an explaination.

In [25]:
# Testing LLM
system_prompt = "You are a helpful assistant."
input_query = "Hi"
model_name = "llama3.3:70b-instruct-q6_K"

In [26]:
%%time

response = llm(model_name, system_prompt, input_query)
explaination = response['response']
print(explaination)

Response received successfully:
Hello! How can I assist you today? Do you have any questions or need help with something in particular? I'm here to help!
CPU times: total: 0 ns
Wall time: 20.2 s


In [27]:
# Picking one row from dataset and converting it into string.
user_input = json.dumps(data[10000])
user_input

'{"post_id": "1178534874602496000_twitter", "tweet_text": "<user> <user> this is exactly the same as the argument that people who get raped when they were dressed scantily were asking for it you are lame af", "key_features": [], "target": "None", "label": "normal"}'

In [28]:
# Testing LLM

system_prompt = """
You are supposed to write an explaination to user's input explaining why ITS hate speech and why its not hate speech, based on the information provided. Our annotators has already decided if a text is a hate speech or not, and labeled.
you will be given
tweet_text : 

key_features: 

{'post_id': ## post id . ignore it
 'tweet_text': #its the main user text that you need to write explaination on.
 'key_features': [] # list of the words that made a decision, important feature
 'target': # targetted audience. 
 'label': # either offensive, hate speech or normal.   
 }

 If label is neutral and you think the text is hate speech. you still have to explain why this text is not hate speech. 
 if the label is offensive, you judge how its offensive based on text and keyfeatures etc.
 
# Notes:
1. Explaination should be one paragraph
2. Explaination should explain the context and user's intention.
3. How is the explaination bad, and how is it not.
4. what helps in making the decision.

Now you will be given user input and you have to Write explainations.
"""

input_query = user_input
model_name = "llama3.3:70b-instruct-q6_K"

response = llm(model_name, system_prompt, input_query)
explaination = response['response']
print(explaination)

Response received successfully:
The given tweet text, although labeled as "normal," can be perceived as both hate speech and not hate speech depending on the context and interpretation. On one hand, the language used is strong, with the phrase "you are lame af" being derogatory, which could imply a form of verbal aggression or insult towards an individual or group. This kind of language might be seen as offensive or even hateful by some standards because it directly attacks the character or intelligence of the person being addressed. On the other hand, the tweet's core argument seems to critique and liken an unspecified argument to a harmful and victim-blaming attitude often applied to rape victims based on their attire, suggesting that the author is actually advocating against such harmful attitudes and defending victims' rights. The absence of specific key features listed might indicate that the annotators focused more on the intent behind the message rather than its literal content,

In [11]:
# for a in data.items():
#     print(a.key())
#     break
# # a[1].keys()

In [12]:
# a[1]['annotators']

In [13]:
# a[1]

# Deploy

In [20]:
index_name = "HateXplain_cpu_test"

In [21]:
create_index(index_name)

In [22]:
print(qdrant_client.get_collections())

collections=[CollectionDescription(name='CUDA'), CollectionDescription(name='test_index'), CollectionDescription(name='HateXplain_8129'), CollectionDescription(name='HateXplain_cpu_test'), CollectionDescription(name='LnR')]


In [23]:
## Upload data into index

In [24]:
def upsert_in_qdrant_collection(data_list, data_list_embeddings, ids):
    try:
        qdrant_client.upsert(
            collection_name=index_name,
            points=Batch(
                ids=ids,
                vectors=data_list_embeddings,
                payloads=data_list
            ),
        )
    except Exception as e:
        # traceback.print_exc()
        print(f"Exception in create_embeddings_and_upsert {e}")

In [25]:
def create_embeddings_and_upsert(batch_size=1000):

    print("Creating rows embeddings")
    ids = list(range(1, len(data) + 1))
    
    for i in tqdm(range(0, len(data), batch_size)):
        batch_data = data[i:i + batch_size]
        batch_ids = ids[i:i + batch_size]
        
        batch_data_list_embeddings = []
        for row in batch_data:
            payload = (
                f"tweet_text: {row['tweet_text']}\n"
                f"key_features: {row['key_features']}\n"
                f"target: {row['target']}\n"
                f"label: {row['label']}\n"
                # WE add EXPLAINATIONS HERE
                # f"post_id: {row['post_id']}"
            )
            
            batch_data_list_embeddings.append(EMBEDDINGS_MODEL.encode(payload))
            #batch_ids.append(row['post_id'])

        # Call the Qdrant function
        upsert_in_qdrant_collection(batch_data, batch_data_list_embeddings, batch_ids)

In [ ]:
create_embeddings_and_upsert(batch_size=200)

Creating rows embeddings


  2%|██▊                                                                                                                                              | 2/101 [03:15<2:42:18, 98.37s/it]

In [1]:
file_size = 322663509  # bytes
limit = 33554432  # bytes

# Calculate percentage larger
percentage_larger = ((file_size - limit) / limit) * 100
percentage_larger

861.6121917963028

## Query RAG

In [55]:
user_input = "loomergate"
top_k = 3
embedding = EMBEDDINGS_MODEL.encode(user_input)

In [56]:
search_result = qdrant_client.search(
    collection_name=index_name, query_vector=embedding, limit=5
)
search_result

[ScoredPoint(id=17182, version=171, score=918.36975, payload={'post_id': '22049484_gab', 'tweet_text': 'is there any opposition to muslim invasion in sweden', 'key_features': [], 'target': 'Islam', 'label': 'normal'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=16153, version=161, score=892.71375, payload={'post_id': '18469603_gab', 'tweet_text': 'how boomers let jews loot the usa', 'key_features': [], 'target': 'Jewish, Men, None, Women', 'label': 'normal'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=17452, version=174, score=876.84314, payload={'post_id': '22918483_gab', 'tweet_text': 'reminder that google is run by jews', 'key_features': [], 'target': 'Jewish, None', 'label': 'normal'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=11898, version=118, score=873.79474, payload={'post_id': '1167982348769660928_twitter', 'tweet_text': 'everyone in your life is a fucking lie i promise', 'key_features': [], 'target': 'Men, None, 

In [33]:
user_input = "I might rape console players tonight"
top_k = 5
embedding = EMBEDDINGS_MODEL.encode(user_input)
search_result = qdrant_client.search(
    collection_name=index_name, query_vector=embedding, limit=top_k
)
search_result

[ScoredPoint(id=8179, version=41, score=13764.598, payload={'post_id': '1179085583680819200_twitter', 'tweet_text': 'please i want to get raped', 'key_features': [], 'target': 'None', 'label': 'normal'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=1275, version=7, score=12908.079, payload={'post_id': '1179064447941402625_twitter', 'tweet_text': 'love getting ass raped by pc kids on pubs when i am on console 🤠', 'key_features': [], 'target': 'Homosexual, None', 'label': 'normal'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=7475, version=38, score=12023.535, payload={'post_id': '1178704722343219200_twitter', 'tweet_text': '<user> i d get raped for a rational debate', 'key_features': [], 'target': 'None', 'label': 'normal'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=13476, version=68, score=12018.386, payload={'post_id': '1242254807164948481_twitter', 'tweet_text': '<user> <user> <number> less boys being raped', 'key_features'